In [1]:
import pandas as pd

df = pd.read_csv("BarcelonaAbnb_limpio.csv")
df
import numpy as np

df["log_price"] = np.log1p(df["price"])


In [2]:
y_reg = df["price"].values
y_reg


array([210., 285., 170., ...,  90.,  90., 120.])

In [3]:
cols_excluir_reg = [
    "price",
    "estimated_revenue_l365d"
    ,  "log_price"
]

X_reg = df.drop(columns=cols_excluir_reg)
X_reg = X_reg.select_dtypes(include=["number"])

X_reg 


,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,...,room_type_Shared room,neighbourhood_group_cleansed_Eixample,neighbourhood_group_cleansed_Gràcia,neighbourhood_group_cleansed_Horta-Guinardó,neighbourhood_group_cleansed_Les Corts,neighbourhood_group_cleansed_Nou Barris,neighbourhood_group_cleansed_Sant Andreu,neighbourhood_group_cleansed_Sant Martí,neighbourhood_group_cleansed_Sants-Montjuïc,neighbourhood_group_cleansed_Sarrià-Sant Gervasi
0,0.0,96.0,91.0,0.0,1.0,1.0,41.405560,2.172620,8.0,2.0,...,0,1,0,0,0,0,0,0,0,0
1,0.0,100.0,96.0,1.0,1.0,1.0,41.412432,2.219750,5.0,2.0,...,0,0,0,0,0,0,0,1,0,0
2,0.0,100.0,100.0,0.0,1.0,1.0,41.405660,2.170150,6.0,2.0,...,0,0,1,0,0,0,0,0,0,0
3,0.0,100.0,92.0,0.0,1.0,1.0,41.380620,2.175170,2.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,100.0,92.0,0.0,1.0,1.0,41.379780,2.176230,9.0,3.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13410,1.0,100.0,98.0,0.0,1.0,1.0,41.382010,2.177120,5.0,1.0,...,0,0,0,0,0,0,0,0,0,0
13411,1.0,87.0,6.0,0.0,1.0,1.0,41.410689,2.170413,3.0,1.0,...,0,0,0,1,0,0,0,0,0,0
13412,1.0,87.0,6.0,0.0,1.0,1.0,41.390695,2.164782,4.0,1.0,...,0,1,0,0,0,0,0,0,0,0
13413,1.0,87.0,6.0,0.0,1.0,1.0,41.396987,2.166294,4.0,1.0,...,0,1,0,0,0,0,0,0,0,0


In [4]:
from sklearn.model_selection import train_test_split

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg,
    test_size=0.2,
    random_state=42
)


In [5]:
from sklearn.preprocessing import StandardScaler
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)


In [6]:
from tensorflow import keras
from tensorflow.keras import layers

def build_regression_model(
    n_features,
    n_hidden1=128,
    n_hidden2=64,
    n_hidden3=32,
    dropout_rate=0.2,
    learning_rate=1e-3
):
    model = keras.Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(n_hidden1, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden2, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden3, activation="relu"),
        layers.Dense(1, activation="linear") 
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    return model


In [7]:
n_features_reg = X_train_reg_scaled.shape[1]

model_reg = build_regression_model(n_features_reg)

history_reg = model_reg.fit(
    X_train_reg_scaled,
    y_train_reg,
    validation_data=(X_test_reg_scaled, y_test_reg),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 7920.7202 - mae: 61.9420 - val_loss: 3294.4536 - val_mae: 41.5545
Epoch 2/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 3343.0090 - mae: 41.4257 - val_loss: 3047.8523 - val_mae: 39.8251
Epoch 3/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3190.8374 - mae: 40.3063 - val_loss: 2945.4128 - val_mae: 39.2439
Epoch 4/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3099.5886 - mae: 39.6020 - val_loss: 2870.8979 - val_mae: 38.7652
Epoch 5/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 3041.2659 - mae: 39.1992 - val_loss: 2845.3447 - val_mae: 38.1192
Epoch 6/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2946.1748 - mae: 38.5999 - val_loss: 2841.6399 - val_mae: 38.1825
Epoch 7/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2906.1726 - mae: 38.2551 - val_loss: 2755.5393 - val_mae: 37.9906
Epoch 8/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2862.6997 - mae: 37.9783 - val_loss: 2743.7893 - v

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

y_pred_reg = model_reg.predict(X_test_reg_scaled).flatten()

mse = mean_squared_error(y_test_reg, y_pred_reg)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_reg, y_pred_reg)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)



84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
MSE: 2272.0591479683226
RMSE: 47.66612159561886
MAE: 32.84133217008621


In [9]:
df["price"].describe()

eval = rmse/df["price"].mean()
print("Evaluation (RMSE / Mean Price):", eval)


Evaluation (RMSE / Mean Price): 0.33078065807638457


Hicimos research y vimos que transformar logarítmicamente el precio estabiliza la varianza y reduce la influencia de outliers, lo que nos puede servir para predecir de mejor forma. Ahora predeciremos con base en esto.

In [10]:
y_reg = df["log_price"].values


In [11]:
cols_excluir_reg = [
    "price",
    "log_price",
    "estimated_revenue_l365d"  
]

X_reg = df.drop(columns=cols_excluir_reg)
X_reg = X_reg.select_dtypes(include=["number"])


In [12]:
from sklearn.model_selection import train_test_split

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg,
    test_size=0.2,
    random_state=42
)


In [13]:
from sklearn.preprocessing import StandardScaler

scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)


In [14]:
from tensorflow import keras
from tensorflow.keras import layers

def build_regression_model(
    n_features,
    n_hidden1=128,
    n_hidden2=64,
    n_hidden3=32,
    dropout_rate=0.2,
    learning_rate=1e-3
):
    model = keras.Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(n_hidden1, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden2, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden3, activation="relu"),
        layers.Dense(1, activation="linear")
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    return model


In [15]:
n_features_reg = X_train_reg_scaled.shape[1]

model_reg_log = build_regression_model(n_features_reg)

history_reg_log = model_reg_log.fit(
    X_train_reg_scaled,
    y_train_reg,
    validation_data=(X_test_reg_scaled, y_test_reg),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.7833 - mae: 0.9432 - val_loss: 0.4588 - val_mae: 0.5462
Epoch 2/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5272 - mae: 0.5732 - val_loss: 0.2895 - val_mae: 0.4217
Epoch 3/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4050 - mae: 0.4993 - val_loss: 0.2569 - val_mae: 0.3977
Epoch 4/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3334 - mae: 0.4548 - val_loss: 0.2470 - val_mae: 0.3986
Epoch 5/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2786 - mae: 0.4178 - val_loss: 0.2831 - val_mae: 0.4411
Epoch 6/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2460 - mae: 0.3909 - val_loss: 0.2358 - val_mae: 0.3952
Epoch 7/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2115 - mae: 0.3620 - val_loss: 0.1650 - val_mae: 0.3153
Epoch 8/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1923 - mae: 0.3430 - val_loss: 0.1683 - val_mae: 0.3207
Epoch 9/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/

In [16]:
y_pred_log = model_reg_log.predict(X_test_reg_scaled).flatten()

# Convertir log_price → price
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test_reg)


84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)

MSE: 2385.367015378141
RMSE: 48.84021923966088
MAE: 32.31275649964698


In [18]:
import mlflow
import mlflow.keras

mlflow.set_tracking_uri("http://localhost:5000") 
experiment_reg = mlflow.set_experiment("airbnb-barcelona-reg-nn")
print("Experiment ID (reg):", experiment_reg.experiment_id)



Experiment ID (reg): 614581612292287758


In [19]:
from tensorflow import keras
from tensorflow.keras import layers

def build_regression_model(
    n_features,
    n_hidden1=128,
    n_hidden2=64,
    n_hidden3=32,
    dropout_rate=0.2,
    learning_rate=1e-3
):
    model = keras.Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(n_hidden1, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden2, activation="relu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden3, activation="relu"),
        layers.Dense(1, activation="linear")  # regresión
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    return model


In [20]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def run_mlflow_reg(
    n_hidden1=128,
    n_hidden2=64,
    n_hidden3=32,
    dropout_rate=0.2,
    learning_rate=1e-3,
    epochs=80,
    batch_size=32,
    run_name="reg_base",
    experiment_id=None
):
    n_features_reg = X_train_reg_scaled.shape[1]

    with mlflow.start_run(run_name=run_name, experiment_id=experiment_id):

        # Log hiperparámetros
        mlflow.log_param("n_hidden1", n_hidden1)
        mlflow.log_param("n_hidden2", n_hidden2)
        mlflow.log_param("n_hidden3", n_hidden3)
        mlflow.log_param("dropout_rate", dropout_rate)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)

        # Construir modelo
        model = build_regression_model(
            n_features=n_features_reg,
            n_hidden1=n_hidden1,
            n_hidden2=n_hidden2,
            n_hidden3=n_hidden3,
            dropout_rate=dropout_rate,
            learning_rate=learning_rate
        )

        # Entrenar
        history = model.fit(
            X_train_reg_scaled,
            y_train_reg,
            validation_data=(X_test_reg_scaled, y_test_reg),
            epochs=epochs,
            batch_size=batch_size,
            verbose=0
        )

        # Predicción en escala log
        y_pred_log = model.predict(X_test_reg_scaled).flatten()

        # Volver a precio real (euros)
        y_true = np.expm1(y_test_reg)
        y_pred = np.expm1(y_pred_log)

        # Métricas reales
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        mean_price = y_true.mean()
        rel_rmse = rmse / mean_price

        # Log métricas
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("rmse_over_mean_price", rel_rmse)

        # Guardar modelo
        mlflow.keras.log_model(model, "model")

        print(f"Run '{run_name}' -> RMSE={rmse:.2f}, MAE={mae:.2f}, RMSE/Mean={rel_rmse:.3f}")


In [21]:
#estas configs se demoran mucho
"""
configs_reg = [
    # ---- MODELOS PEQUEÑOS ----
    {"run_name": "reg_small_1_500ep", "n_hidden1": 32,  "n_hidden2": 16,  "n_hidden3": 8,   "learning_rate": 1e-3, "dropout_rate": 0.1, "epochs": 250,  "batch_size": 32},

    {"run_name": "reg_small_3_500ep", "n_hidden1": 64,  "n_hidden2": 32,  "n_hidden3": 16,  "learning_rate": 1e-3, "dropout_rate": 0.2, "epochs": 250,  "batch_size": 64},
    {"run_name": "reg_small_4_500ep", "n_hidden1": 16,  "n_hidden2": 8,   "n_hidden3": 4,   "learning_rate": 1e-3, "dropout_rate": 0.0, "epochs": 250,  "batch_size": 32},
    {"run_name": "reg_small_5_500ep", "n_hidden1": 16,  "n_hidden2": 8,   "n_hidden3": 4,   "learning_rate": 5e-4, "dropout_rate": 0.1, "epochs": 250,  "batch_size": 32},
    # ---- MODELOS MEDIOS ----
    {"run_name": "reg_medium_1_500ep", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 1e-3, "dropout_rate": 0.2, "epochs": 250,  "batch_size": 32},
    {"run_name": "reg_medium_2_500ep", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 5e-4, "dropout_rate": 0.3, "epochs": 250,  "batch_size": 32},
    {"run_name": "reg_medium_3_500ep", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64, "learning_rate": 1e-3, "dropout_rate": 0.2, "epochs": 250,  "batch_size": 64},
    {"run_name": "reg_medium_4_500ep", "n_hidden1": 64,  "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 3e-4, "dropout_rate": 0.2, "epochs": 250,  "batch_size": 32},
    {"run_name": "reg_medium_5_500ep", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 1e-4, "dropout_rate": 0.2, "epochs": 250, "batch_size": 32},

    # ---- MODELOS GRANDES ----
    {"run_name": "reg_large_1_500ep", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64,  "learning_rate": 1e-3, "dropout_rate": 0.3, "epochs": 250, "batch_size": 32},
    {"run_name": "reg_large_2_500ep", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64,  "learning_rate": 5e-4, "dropout_rate": 0.3, "epochs": 250, "batch_size": 32},
    {"run_name": "reg_large_3_500ep", "n_hidden1": 512, "n_hidden2": 256, "n_hidden3": 128, "learning_rate": 3e-4, "dropout_rate": 0.3, "epochs": 250, "batch_size": 64},
    {"run_name": "reg_large_4_500ep", "n_hidden1": 512, "n_hidden2": 256, "n_hidden3": 128, "learning_rate": 1e-4, "dropout_rate": 0.4, "epochs": 250, "batch_size": 32},
    {"run_name": "reg_large_5_500ep", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64,  "learning_rate": 2e-4, "dropout_rate": 0.3, "epochs": 250, "batch_size": 32},

    # ---- VARIANDO LEARNING RATE ----
    {"run_name": "reg_lr_1e_minus2_500ep", "n_hidden1": 64,  "n_hidden2": 32,  "n_hidden3": 16, "learning_rate": 1e-2, "dropout_rate": 0.2, "epochs": 250,  "batch_size": 32},
    {"run_name": "reg_lr_5e_minus3_500ep", "n_hidden1": 64,  "n_hidden2": 32,  "n_hidden3": 16, "learning_rate": 5e-3, "dropout_rate": 0.2, "epochs": 250,  "batch_size": 32},
    {"run_name": "reg_lr_3e_minus4_500ep", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 3e-4, "dropout_rate": 0.2, "epochs": 250, "batch_size": 32},
    {"run_name": "reg_lr_1e_minus4_500ep", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 1e-4, "dropout_rate": 0.3, "epochs": 250, "batch_size": 32},
    {"run_name": "reg_lr_5e_minus5_500ep", "n_hidden1": 256, "n_hidden2": 128, "n_hidden3": 64, "learning_rate": 5e-5, "dropout_rate": 0.2, "epochs": 250, "batch_size": 32},
]
"""

'\nconfigs_reg = [\n    # ---- MODELOS PEQUEÑOS ----\n    {"run_name": "reg_small_1_500ep", "n_hidden1": 32,  "n_hidden2": 16,  "n_hidden3": 8,   "learning_rate": 1e-3, "dropout_rate": 0.1, "epochs": 250,  "batch_size": 32},\n\n    {"run_name": "reg_small_3_500ep", "n_hidden1": 64,  "n_hidden2": 32,  "n_hidden3": 16,  "learning_rate": 1e-3, "dropout_rate": 0.2, "epochs": 250,  "batch_size": 64},\n    {"run_name": "reg_small_4_500ep", "n_hidden1": 16,  "n_hidden2": 8,   "n_hidden3": 4,   "learning_rate": 1e-3, "dropout_rate": 0.0, "epochs": 250,  "batch_size": 32},\n    {"run_name": "reg_small_5_500ep", "n_hidden1": 16,  "n_hidden2": 8,   "n_hidden3": 4,   "learning_rate": 5e-4, "dropout_rate": 0.1, "epochs": 250,  "batch_size": 32},\n    # ---- MODELOS MEDIOS ----\n    {"run_name": "reg_medium_1_500ep", "n_hidden1": 128, "n_hidden2": 64,  "n_hidden3": 32, "learning_rate": 1e-3, "dropout_rate": 0.2, "epochs": 250,  "batch_size": 32},\n    {"run_name": "reg_medium_2_500ep", "n_hidden1": 

In [22]:
"""
for cfg in configs_reg:
    print(f"=== Corriendo {cfg['run_name']} ===")
    run_mlflow_reg(**cfg, experiment_id=experiment_reg.experiment_id)
    print()
"""

'\nfor cfg in configs_reg:\n    print(f"=== Corriendo {cfg[\'run_name\']} ===")\n    run_mlflow_reg(**cfg, experiment_id=experiment_reg.experiment_id)\n    print()\n'

Haremos el mismo modelo pero con ELU porque según chat es mejor :D


In [23]:
from tensorflow import keras
from tensorflow.keras import layers

def build_regression_model_elu(
    n_features,
    n_hidden1=128,
    n_hidden2=64,
    n_hidden3=32,
    dropout_rate=0.2,
    learning_rate=1e-3
):
    model = keras.Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(n_hidden1, activation="elu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden2, activation="elu"),
        layers.Dropout(dropout_rate),
        layers.Dense(n_hidden3, activation="elu"),
        layers.Dense(1, activation="linear")  # regresión real
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    return model


In [24]:
n_features_reg = X_train_reg_scaled.shape[1]

model_reg_elu = build_regression_model_elu(n_features_reg)

history_reg_elu = model_reg_elu.fit(
    X_train_reg_scaled,
    y_train_reg,
    validation_data=(X_test_reg_scaled, y_test_reg),
    epochs=100,
    batch_size=32,
    verbose=1
)


Epoch 1/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 2.0110 - mae: 0.9970 - val_loss: 0.2467 - val_mae: 0.3893
Epoch 2/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3729 - mae: 0.4713 - val_loss: 0.1768 - val_mae: 0.3246
Epoch 3/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.2306 - mae: 0.3744 - val_loss: 0.1803 - val_mae: 0.3341
Epoch 4/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 0.1893 - mae: 0.3388 - val_loss: 0.1603 - val_mae: 0.3098
Epoch 5/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.1750 - mae: 0.3251 - val_loss: 0.1515 - val_mae: 0.2958
Epoch 6/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1696 - mae: 0.3182 - val_loss: 0.1528 - val_mae: 0.2988
Epoch 7/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1616 - mae: 0.3107 - val_loss: 0.1520 - val_mae: 0.2962
Epoch 8/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1586 - mae: 0.3066 - val_loss: 0.1452 - val_mae: 0.2908
Epoch 9/100
336/336 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/

In [25]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred_log = model_reg_elu.predict(X_test_reg_scaled).flatten()

# Volver al precio real
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test_reg)

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)

mean_price = y_true.mean()
rel_rmse = rmse / mean_price

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("RMSE / mean_price:", rel_rmse)


84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
MSE: 2223.43232598581
RMSE: 47.15328542091007
MAE: 31.862441858781438
RMSE / mean_price: 0.32785050632910856


In [29]:
p99 = df["price"].quantile(0.99)
print("Antes:", df.shape)
df_filt = df[df["price"] <= p99]




print("Después:", df_filt.shape) 
df_filt["log_price"] = np.log1p(df_filt["price"])



cols_excluir_reg = [
    "price",
    "log_price",
    "estimated_revenue_l365d"  # si existe
]


y_reg = df_filt["log_price"].values

X_reg = df_filt.drop(columns=cols_excluir_reg)
X_reg = X_reg.select_dtypes(include=["number"])

from sklearn.model_selection import train_test_split

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg,
    test_size=0.2,
    random_state=42
)


from sklearn.preprocessing import StandardScaler

scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

n_features_reg = X_train_reg_scaled.shape[1]
model_reg_elu = build_regression_model_elu(n_features_reg)

history_reg_elu = model_reg_elu.fit(
    X_train_reg_scaled,
    y_train_reg,
    validation_data=(X_test_reg_scaled, y_test_reg),
    epochs=100,
    batch_size=32,
    verbose=1
)



Antes: (13415, 49)
Después: (13282, 49)
Epoch 1/100


C:\Users\jo_ga\AppData\Local\Temp\ipykernel_19388\1489508282.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filt["log_price"] = np.log1p(df_filt["price"])


333/333 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.9751 - mae: 0.9792 - val_loss: 0.2079 - val_mae: 0.3504
Epoch 2/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3328 - mae: 0.4518 - val_loss: 0.1715 - val_mae: 0.3200
Epoch 3/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2190 - mae: 0.3671 - val_loss: 0.1757 - val_mae: 0.3226
Epoch 4/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1895 - mae: 0.3411 - val_loss: 0.1522 - val_mae: 0.2993
Epoch 5/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1715 - mae: 0.3222 - val_loss: 0.1495 - val_mae: 0.2924
Epoch 6/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1673 - mae: 0.3165 - val_loss: 0.1669 - val_mae: 0.3153
Epoch 7/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1616 - mae: 0.3123 - val_loss: 0.1457 - val_mae: 0.2858
Epoch 8/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1568 - mae: 0.3061 - val_loss: 0.1513 - val_mae: 0.2958
Epoch 9/100
333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss:

In [30]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred_log = model_reg_elu.predict(X_test_reg_scaled).flatten()

y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test_reg)

mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)

mean_price = y_true.mean()
rel_rmse = rmse / mean_price

print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("RMSE / mean_price:", rel_rmse)


84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
MSE: 2333.4818210825288
RMSE: 48.30612612373847
MAE: 31.882638967167313
RMSE / mean_price: 0.33701299511288907
